# Chapter 2 — An Array of Sequences

**Sections with code snippets in this chapter:**

- [List Comprehensions and Generator Expressions](#List-Comprehensions-and-Generator-Expressions)
- [Tuples Are Not Just Immutable Lists](#Tuples-Are-Not-Just-Immutable-Lists)
- [Unpacking sequences and iterables](#Unpacking-sequences-and-iterables)
- [Pattern Matching with Sequences](#Pattern-Matching-with-Sequences)
- [Slicing](#Slicing)
- [Using + and \* with Sequences](#Using-+-and-*-with-Sequences)
- [Augmented Assignment with Sequences](#Augmented-Assignment-with-Sequences)
- [list.sort and the sorted Built-In Function](#list.sort-and-the-sorted-Built-In-Function)
- [When a List Is Not the Answer](#When-a-List-Is-Not-the-Answer)
- [Memory Views](#Memory-Views)
- [NumPy and SciPy](#NumPy-and-SciPy)
- [Deques and Other Queues](#Deques-and-Other-Queues)
- [Soapbox](#Soapbox)


# Overview of Built-In Sequences

Sequence: ordered collection, Each element in a sequence has a unique index that distinguishes it from others in the sequence.


#### Grouping By Container and Flat

- Container sequences: Container sequences: A container sequence holds references to the objects it contains, can hold elements of different types, including other sequences or containers. flexible, allow to store a variety of objects within them
  - list
  - tuple
  - collections.deque, ...
- Flat sequences (homogeneous sequences): A flat sequence stores the value of its contents in its own memory space, and not as distinct Python objects
  - str
  - bytes
  - array.array, ...
  ![Alt text](https://programming.vip/images/doc/5e98fd2450631cfe68f385f5707b2f0e.jpg "Flat Vs Container")


#### Grouping By Mutable and Immutable

- Mutable sequences
  - list
  - bytearray
  - array.array
  - collections.deque
- Immutable sequences
  - tuple
  - str
  - bytes


## List Comprehensions and Generator Expressions


`quick way to build a sequence is using a listcomps (List Comprehntions)`

- more readable ???
- often faster ???


In [ ]:
symbols = "$¢£¥€¤"

codes = []
for symbol in symbols:
    codes.append(ord(symbol))
print(codes)

codes = [ord(symbol) for symbol in symbols]
print(codes)

> ### Tip
>
> In Python code, line breaks are ignored inside pairs of [], {}, or ().  
> also trailing comma will be ignored. so it is thoughtful to put a comma after the last item.


#### LOCAL SCOPE WITHIN COMPREHENSIONS AND GENERATOR EXPRESSIONS


list comprehensions, generator expressions, and their siblings set and dict comprehensions have a local scope to hold the variables assigned in the for clause.


In [ ]:
x = "ABC"

codes = [ord(i) for i in x]
print(i)

In [ ]:
codes = []
for i in x:
    codes.append(i)
print(i)

In [9]:
x = "ABC"

codes = [last := ord(i) for i in x]
print(last)

67


#### Listcomps Versus map and filter


In [ ]:
symbols = "$¢£¥€¤"
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

In [ ]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

In [11]:
import timeit

TIMES = 1_000_000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""


def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *(f"{x:.3f}" for x in res))


clock("listcomp        :", "[ord(s) for s in symbols if ord(s) > 127]")
clock("listcomp + func :", "[ord(s) for s in symbols if non_ascii(ord(s))]")
clock("filter + lambda :", "list(filter(lambda c: c > 127, map(ord, symbols)))")
clock("filter + func   :", "list(filter(non_ascii, map(ord, symbols)))")

listcomp        : 0.432 0.433 0.484 0.443 0.445
listcomp + func : 0.689 0.687 0.682 0.661 0.673
filter + lambda : 0.597 0.584 0.598 0.580 0.598
filter + func   : 0.565 0.547 0.568 0.554 0.538


#### Cartesian Products List Comprehension (Nested)


In [ ]:
colors = ["black", "white"]
sizes = ["S", "M", "L"]
tshirts = [(color, size) for color in colors for size in sizes]

print(tshirts)

In [ ]:
for color in colors:
    for size in sizes:
        print((color, size))

In [ ]:
shirts = [(color, size) for size in sizes for color in colors]
tshirts

## Generator Expressions

- save memory
- enclosed in parentheses
- in single argument positions in functions ther is no need to duplicate the enclosing parentheses.


In [ ]:
symbols = "$¢£¥€¤"
tuple(ord(symbol) for symbol in symbols)

In [ ]:
import array

array.array("I", (ord(symbol) for symbol in symbols))

#### Example 2-6. Cartesian product in a generator expression


In [ ]:
colors = ["black", "white"]
sizes = ["S", "M", "L"]

for tshirt in ("%s %s" % (c, s) for c in colors for s in sizes):
    print(tshirt)

## Tuples Are Not Just Immutable Lists

- can used as a:
  - immutable lists
  - records with no field name


#### Tuples as Records


In [ ]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ("Tokyo", 2003, 32_450, 0.66, 8014)
traveler_ids = [("USA", "31195855"), ("BRA", "CE342567"), ("ESP", "XDA205856")]

for passport in sorted(traveler_ids):
    print("%s/%s" % passport)

In [ ]:
for country, _ in traveler_ids:
    print(country)

> ---
>
> ### Tip
>
> Using \_ as a dummy variable is just a convention
>
> 1. In the Python console, the result of executing a line is assigned to \_ unless the result is None.
> 2. ## In a match/case statement, \_ is a wildcard that matches any value but is never assigned a value


### Tuples as Immutable Lists


Two key benefits of using Tuples as Immutable Lists

- Clarity: when you see a tuple in code, you know its length will never change
- Performance: a tuple uses less memory than a list of the same length, and they allow Python to do some optimizations


be aware that the immutability of a tuple only applies to the references contained in it. References in a tuple cannot be deleted or replaced. But if one of those references points to a mutable object, and that object is changed, then the value of the tuple changes.


In [1]:
a = (10, "alpha", [1, 2])
b = (10, "alpha", [1, 2])
a == b

True

In [2]:
b[-1].append(99)
a == b

False

an object is only hashable if its value cannot ever change: #edited


In [ ]:
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True


tf = (10, "alpha", (1, 2))  # Contains no mutable items
tm = (10, "alpha", [1, 2])  # Contains a mutable item (list)
fixed(tf)

In [ ]:
fixed(tm)

#### Are tuples more efficient than lists in Python?

- the Python compiler generates bytecode for a tuple constant in one operation; but for a list lteral, the generated bytecode pushes each element as a separate constant to the data stack, and then builds the list.
- tuple(t) returns reference to `t`. but list(t) return a new copy of `t`.
- tuple is allocated the exact memory space it need. but list allocated with room to spare, to mortize the cost of future appends
- The references to the items in a tuple are stored in an array in the tuple struct, while a list holds a pointer to an array of references stored elsewhere. The indirection is necessary because when a list grows beyond the space currently allocated, Python needs to reallocate the array of references to make room. The extra indirection makes CPU caches less effective.


## Unpacking sequences and iterables


unpacking works with any iterable object as the data source including iterators which don’t support index notation


In [ ]:
a, *_, end = (i for i in range(10))

In [3]:
t = (20, 8)
quotient, remainder = divmod(*t)
# takes two arguments and returns a tuple containing the quotient and remainder of the division operation.

print(f"{quotient=}\n{remainder=}")

quotient=2
remainder=4


In [4]:
import os

_, filename = os.path.split("/home/luciano/.ssh/id_rsa.pub")
filename

'id_rsa.pub'

### Using \* to grab excess items


In [ ]:
a, b, *rest = range(5)
a, b, rest

In [6]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [5]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

### Unpacking with \* in function calls and sequence literals


In [ ]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest


fun(*[1, 2], 3, *range(4, 7))

In [9]:
*range(4), 4

(0, 1, 2, 3, 4)

### Nested unpacking


In [14]:
metro_areas = [
    ("Tokyo", "JP", 36.933, (35.689722, 139.691667)),
    ("Delhi NCR", "IN", 21.935, (28.613889, 77.208889)),
]
for name, _, _, (lat, lon) in metro_areas:
    print(f"{name} | {lat:9.4f} | {lon:9.4f}")

hey
Tokyo |   35.6897 |  139.6917
Delhi NCR |   28.6139 |   77.2089


## Pattern Matching with Sequences

Python pattern matching is a feature that allows for concise and readable code by matching the structure of data against patterns and executing corresponding code blocks.


In [1]:
# edited
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age


def handle_command(data):
    match data:
        case {
            "type": "person",
            "name": str(name),
            "age": int(age),
        } if age >= 18:  # runtime type check: first item must be an instance of str
            print("type: dict, handling...")

        case int():
            print("type: int, handling...")

        case [
            str(name),
            *_,
            (float(lat), float(lon)),
        ]:  # *_ matches any number of items, without binding them to a variable
            print("type: list, handling...")

        case Person(name=str(name), age=int(age)) as person:
            print("type: Person", name, age, person)

        case _:
            raise ValueError(f"Value Not Match: {data}")


handle_command({"type": "person", "name": "John", "age": 25})
handle_command(234)
handle_command(Person(name="sadegh", age=18))
handle_command(["username", 20, (int(12345), int(54321))])

type: dict, handling...
type: int, handling...
type: Person sadegh 18 <__main__.Person object at 0x7f40024701f0>


ValueError: Value Not Match: ['username', 20, (12345, 54321)]

#### edited

#### Without a catch-all, the whole match statement does nothing when a subject does not match any case—and this can be a silent failure.


### Pattern Matching Sequences in an Interpreter

#### This section compares two implementations of a Lisp interpreter: lis.py and an updated version using Python 3.10's match/case syntax. The comparison highlights the benefits of using pattern matching in Python code.

The interpreter evaluates lists of expressions, such as calling functions with arguments or defining new functions, and returns the value of the last expression in the body of a function.

#### Example 2-11. Matching patterns without match/case.

[02-array-seq/lispy/py3.9/lis.py](02-array-seq/lispy/py3.9/lis.py)


#### Example 2-12. Pattern matching with match/case—requires Python ≥ 3.10.

[02-array-seq/lispy/py3.10/lis.py](02-array-seq/lispy/py3.10/lis.py)


## Slicing

#### Python's sequence types, including list, tuple, and str, support slicing operations that are more powerful than commonly used. This section describes advanced forms of slicing, with implementation in user-defined classes covered in Chapter 12.


### Why Slices and Range Exclude the Last Item

#### The convention of excluding the last item in Python slices and ranges is convenient due to its compatibility with zero-based indexing used in many languages. It allows for easy calculation of slice and range lengths, as well as splitting a sequence into non-overlapping parts at any index. For instance, my_list[:x] and my_list[x:] can be used to split a sequence at index x.


In [1]:
l = [10, 20, 30, 40, 50, 60]

l[:2]  # split at 2

[10, 20]

In [2]:
l[2:]

[30, 40, 50, 60]

In [ ]:
l[:3]  # split at 3

In [ ]:
l[3:]

###### Further reading


### Slice Objects

#### The slice notation s[a:b:c] can be used to specify a stride or step c, which skips items in the resulting slice. The stride can be negative, resulting in items being returned in reverse order.


In [ ]:
s = "bicycle"
s[::3]

In [ ]:
s[::-1]

In [ ]:
s[::-2]

#### The notation a:b:c is only valid within [] when used as the indexing or subscript operator, and it produces a slice object. Python calls seq.**getitem**(slice(start, stop, step)) to evaluate the expression seq[start:stop:step]. Knowing about slice objects is useful for assigning names to slices, making code more readable. This is demonstrated in Example 2-13 for parsing flat-file data.


#### Example 2-13. Line items from a flat-file invoice


In [ ]:
invoice = """
0.....6.................................40........52...55........
1909 Pimoroni PiBrella                      $17.50    3    $52.50
1489 6mm Tactile Switch x20                  $4.95    2    $9.90
1510 Panavise Jr. - PV-201                  $28.00    1    $28.00
1601 PiTFT Mini Kit 320x240                 $34.95    1    $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)

line_items = invoice.split("\n")[2:]

for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

### Multidimensional Slicing and Ellipsis

#### The [] operator can take multiple indexes or slices separated by commas, which are received as a tuple by the **getitem** and **setitem** special methods. The ellipsis is an alias to the Ellipsis object and can be used as a shortcut when slicing arrays of many dimensions in NumPy. These features exist to support user-defined types and extensions such as NumPy. Slices can also be used to change mutable sequences in place.


### Assigning to Slices

#### Slice notation can be used on the left-hand side of an assignment statement or as the target of a del statement to modify mutable sequences in place. This allows for powerful operations such as grafting and excising.


In [5]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [ ]:
del l[5:7]
l

In [ ]:
l[3::2] = [11, 22]
l

By design, this example raises an exception::


In [8]:
l[2:5] = [100]

In [5]:
l[2:5] = [100]
l

[10, 20, 100, 60]

## Using + and \* with Sequences

#### Usually both operands of + must be of the same sequence type, and neither of them is modified, but a new sequence of that same type is created as result of the concatenation


In [9]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [10]:
5 * "abcd"

'abcdabcdabcdabcdabcd'

#### When using the \* operator to repeat a sequence, be cautious when the sequence contains mutable items. The result may not be what you expect, as the repeated items may reference the same object.


### Building Lists of Lists

#### To initialize a list with a certain number of nested lists, use a list comprehension. This is a better approach than using the \* operator, especially when dealing with mutable items.


#### Example 2-14. A list with three lists of length 3 can represent a tic-tac-toe board


In [6]:
board = [["_"] * 3 for i in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [7]:
board[1][2] = "X"
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

#### Example 2-15. A list with three references to the same list is useless


In [ ]:
weird_board = [["_"] * 3] * 3
weird_board

In [ ]:
weird_board[1][2] = "O"
weird_board

#### Explanation


In [8]:
board = []
for i in range(3):
    row = ["_"] * 3
    board.append(row)
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [ ]:
board[2][0] = "X"
board

## Augmented Assignment with Sequences

#### The augmented assignment operators += and \*= behave differently depending on the first operand. If the first operand implements **iadd** or **imul**, the operation is performed in place. Otherwise, a new object is created and assigned to the first operand. This is true for both mutable and immutable sequences. The **iadd** and **imul** special methods are discussed in Chapter 16 of the book.


In [12]:
l = [1, 2, 3]
print(id(l))
l += [1, 2]
print(id(l))

print(l)
idl = id(l)

2251226702144
2251226702144
[1, 2, 3, 1, 2]


In [ ]:
# NBVAL_IGNORE_OUTPUT
idl

In [ ]:
l *= 2
l

In [ ]:
id(l) == idl  # same list

In [ ]:
t = (1, 2, 3)
idt = id(t)

In [ ]:
# NBVAL_IGNORE_OUTPUT
idt

In [ ]:
t *= 2
id(t) == idt  # new tuple

### A += Assignment Puzzler

#### Example 2-16. A riddle


In [15]:
t = (1, 2, [30, 40])

t[2] += [50, 60]  # the operation can be done with t[2].extend([50,60]), without errors

TypeError: 'tuple' object does not support item assignment

#### Example 2-17. The unexpected result: item t2 is changed and an exception is raised


In [3]:
t

(1, 2, [30, 40, 50, 60])

#### Example 2-18. Bytecode for the expression s[a] += b


In [10]:
import dis

dis.dis("s[a] += b")

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


#### The lessons learned from the example are to avoid putting mutable items in tuples, be aware that augmented assignment is not atomic, and to consider inspecting Python bytecode to better understand what is happening in the code.


## list.sort and the sorted Built-In Function

#### The list.sort() method sorts a list in place and returns None to indicate that the original list was changed and no new list was created (In contrast, the built-in function sorted creates a new list and returns it). This convention is used in Python for functions or methods that modify an object in place. The random.shuffle() function is another example of this convention.

###### The convention of returning None to signal in-place changes has a drawback: we cannot cascade calls to those methods.

#### Both list.sort() and sorted() functions have two optional keyword arguments: reverse and key. The reverse argument, if set to True, sorts the items in descending order. The key argument is a one-argument function that is applied to each item to produce its sorting key, allowing for custom sorting criteria. The default values are False for reverse and the identity function for key.


In [ ]:
fruits = ["grape", "raspberry", "apple", "banana"]
sorted(fruits)

In [ ]:
fruits

In [ ]:
sorted(fruits, reverse=True)

In [ ]:
sorted(fruits, key=len)

In [ ]:
sorted(fruits, key=len, reverse=True)

In [ ]:
fruits

In [ ]:
fruits.sort()
fruits

## When a List Is Not the Answer

While the list type is flexible, other data structures may be better suited for specific requirements. For example, arrays are more memory-efficient for handling large amounts of floating-point values, while deques are more efficient for constantly adding and removing items from opposite ends of a list. If your code frequently checks whether an item is present in a collection, consider using a set


### Arrays

If a list only contains numbers, using an array.array is more memory-efficient and supports all mutable sequence operations. Arrays are as lean as C arrays and use a typecode to determine the underlying C type used to store each item, saving memory for large sequences of numbers.


In [ ]:
from array import array
from random import random, seed

seed(10)  # Use seed to make the output consistent

floats = array("d", (random() for i in range(10**7)))
floats[-1]

In [ ]:
with open("floats.bin", "wb") as fp:
    floats.tofile(fp)

In [ ]:
floats2 = array("d")

with open("floats.bin", "rb") as fp:
    floats2.fromfile(fp, 10**7)

floats2[-1]

In [ ]:
floats2 == floats

##### Table 2-3


|                            | List    | Array   |                                                            |
| -------------------------- | ------- | ------- | ---------------------------------------------------------- |
| `s.__add__(s2)`            | &#9679; | &#9679; | `s + s2` -- concatination                                  |
| `s.__iadd__(s2)`           | &#9679; | &#9679; | `s += s2` -- In-place concatenation                        |
| `s.append(e)`              | &#9679; | &#9679; | Append                                                     |
| `s.byteswap()`             |         | &#9679; | Swap bytes of all items in array for endianness conversion |
| `s.clear()`                | &#9679; |         | Delete all items                                           |
| `s.__contains__(e)`        | &#9679; | &#9679; | `e in s`                                                   |
| `s.copy()`                 | &#9679; |         | Shallow copy of the list                                   |
| `s.__copy__()`             |         | &#9679; | Support for copy.copy                                      |
| `s.count(e)`               | &#9679; | &#9679; | Count occurrences of an element                            |
| `s.__deepcopy__()`         |         | &#9679; | Optimized support for copy.deepcopy                        |
| `s.__delitem__(p)`         | &#9679; | &#9679; | Delete item at position p                                  |
| `s.extend(it)`             | &#9679; | &#9679; | Append items from iterable it                              |
| `s.frombytes(b)`           |         | &#9679; | Append items from byte sequence                            |
| `s.fromfile(f, n)`         |         | &#9679; | Append n items from binary file f                          |
| `s.fromlist(l)`            |         | &#9679; | Append items from list                                     |
| `s.__getitem__(p)`         | &#9679; | &#9679; | s[p]                                                       |
| `s.insert(p, e)`           | &#9679; | &#9679; | Find position of first occurrence of e                     |
| `s.index(e)`               | &#9679; | &#9679; | Insert element e before the item at position p             |
| `s.itemsize`               |         | &#9679; | Length in bytes of each array item                         |
| `s.__iter__()`             | &#9679; | &#9679; | Get iterator                                               |
| `s.__len__()`              | &#9679; | &#9679; | len(s)                                                     |
| `s.__mul__(n)`             | &#9679; | &#9679; | `s * n` -- repeated concatenation                          |
| `s.__imul__(n)`            | &#9679; | &#9679; | `s *= n` -- in-place repeated concatenation                |
| `s.__rmul__(n)`            | &#9679; | &#9679; | `n * s` -- revesedrepeated concatenation                   |
| `s.pop([p])`               | &#9679; | &#9679; | Remove and return item at position p                       |
| `s.remove(e)`              | &#9679; | &#9679; | Remove first occurrence of element e by value              |
| `s.reverse()`              | &#9679; | &#9679; | Reverse the order of the items in place                    |
| `s.__reversed__()`         | &#9679; |         | Get iterator to scan items from last to first              |
| `s.__setitem__(p, e)`      | &#9679; | &#9679; | `s[p] = e`-- put e in position p                           |
| `s.sort([key], [reverse])` | &#9679; |         | Sort items in-place                                        |
| `s.tobytes()`              |         | &#9679; | Return items as packed machine values in a bytes object    |
| `s.tofile(f)`              |         | &#9679; | Save items as packed machine values to binary file f       |
| `s.tolist()`               |         | &#9679; | Return items as numeric objects in a list                  |
| `s.typecode`               |         | &#9679; | C type of the items                                        |


### Memory Views


The memoryview class is a shared-memory sequence type that allows handling slices of arrays without copying bytes. It is inspired by NumPy and allows sharing memory between data structures without copying, which is important for large datasets. The memoryview.cast method lets you change the way multiple bytes are read or written as units without moving bits around.


In [ ]:
octets = array("B", range(6))
m1 = memoryview(octets)
m1.tolist()

In [ ]:
m2 = m1.cast("B", [2, 3])
m2.tolist()

In [ ]:
m3 = m1.cast("B", [3, 2])
m3.tolist()

In [ ]:
m2[1, 1] = 22
m3[1, 1] = 33
octets

In [ ]:
numbers = array("h", [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

In [ ]:
memv[0]

In [ ]:
memv_oct = memv.cast("B")
memv_oct.tolist()

In [ ]:
memv_oct[5] = 4
numbers

### NumPy


NumPy is a powerful library for advanced array and matrix operations in scientific computing applications. It implements multi-dimensional, homogeneous arrays and matrix types and provides efficient element-wise operations. SciPy is a library built on top of NumPy that offers many scientific computing algorithms and leverages C and Fortran codebase for fast and reliable performance. Example 2-22 demonstrates some basic operations with two-dimensional arrays using NumPy.


In [ ]:
import numpy as np

a = np.arange(12)
a

In [ ]:
type(a)

In [ ]:
a.shape

In [ ]:
a.shape = 3, 4
a

In [ ]:
a[2]

In [ ]:
a[2, 1]

In [ ]:
a[:, 1]

In [ ]:
a.transpose()

#### Loading, saving, and vectorized operations


In [ ]:
with open("floats-1M-lines.txt", "wt") as fp:
    for _ in range(1_000_000):
        fp.write(f"{random()}\n")

In [ ]:
floats = np.loadtxt("floats-1M-lines.txt")

In [ ]:
floats[-3:]

In [ ]:
floats *= 0.5
floats[-3:]

In [ ]:
from time import perf_counter as pc

t0 = pc()
floats /= 3
(pc() - t0) < 0.01

In [ ]:
np.save("floats-1M", floats)
floats2 = np.load("floats-1M.npy", "r+")
floats2 *= 6

In [ ]:
floats2[-3:]

### Deques and Other Queues


The class collections.deque is a thread-safe double-ended queue that allows for fast inserting and removing from both ends. It is useful for implementing stacks, queues, and keeping a list of "last seen items" with a fixed maximum length. Example 2-23 demonstrates typical operations performed on a deque.


|                            | List    | Deque   |                                                |
| -------------------------- | ------- | ------- | ---------------------------------------------- |
| `s.__add__(s2)`            | &#9679; |         | `s + s2` -- concatination                      |
| `s.__iadd__(s2)`           | &#9679; | &#9679; | `s += s2` -- In-place concatenation            |
| `s.append(e)`              | &#9679; | &#9679; | Append one element to the right (after last)   |
| `s.appendleft(e)`          |         | &#9679; | Append one element to the left (before first)  |
| `s.clear()`                | &#9679; | &#9679; | Delete all items                               |
| `s.__contains__(e)`        | &#9679; |         | `e in s`                                       |
| `s.copy()`                 | &#9679; |         | Shallow copy of the list                       |
| `s.__copy__()`             |         | &#9679; | Support for copy.copy                          |
| `s.count(e)`               | &#9679; | &#9679; | Count occurrences of an element                |
| `s.__delitem__(p)`         | &#9679; | &#9679; | Delete item at position p                      |
| `s.extend(i)`              | &#9679; | &#9679; | Append items from iterable i to the right      |
| `s.extendleft(i)`          |         | &#9679; | Append items from iterable i to the left       |
| `s.__getitem__(p)`         | &#9679; | &#9679; | s[p]                                           |
| `s.insert(p, e)`           | &#9679; |         | Find position of first occurrence of e         |
| `s.index(e)`               | &#9679; |         | Insert element e before the item at position p |
| `s.__iter__()`             | &#9679; | &#9679; | Get iterator                                   |
| `s.__len__()`              | &#9679; | &#9679; | len(s)                                         |
| `s.__mul__(n)`             | &#9679; |         | `s * n` -- repeated concatenation              |
| `s.__imul__(n)`            | &#9679; |         | `s *= n` -- in-place repeated concatenation    |
| `s.__rmul__(n)`            | &#9679; |         | `n * s` -- revesedrepeated concatenation       |
| `s.pop()*`                 | &#9679; | &#9679; | Remove and return last item                    |
| `s.popleft()`              |         | &#9679; | Remove and return first item                   |
| `s.remove(e)`              | &#9679; | &#9679; | Remove first occurrence of element e by value  |
| `s.reverse()`              | &#9679; | &#9679; | Reverse the order of the items in place        |
| `s.__reversed__()`         | &#9679; | &#9679; | Get iterator to scan items from last to first  |
| `s.rotate()`               |         | &#9679; | Move n items from one end to the other         |
| `s.__setitem__(p, e)`      | &#9679; | &#9679; | `s[p] = e`-- put e in position p               |
| `s.sort([key], [reverse])` | &#9679; |         | Sort items in-place                            |

\*a_list.pop(p) allows removing from position p


#### Example 2-23. Working with a deque


In [ ]:
import collections

dq = collections.deque(range(10), maxlen=10)
dq

In [ ]:
dq.rotate(3)
dq

In [ ]:
dq.rotate(-4)
dq

In [ ]:
dq.appendleft(-1)
dq

In [ ]:
dq.extend([11, 22, 33])
dq

In [ ]:
dq.extendleft([10, 20, 30, 40])
dq

## Soapbox


the creator of Python, highlighted the similarities and differences between ABC compounds and Python tuples. While compounds lacked the ability to be iterated or accessed by index, tuples were made to behave as sequences, contributing to Python's success. The main use case for tuples as records is not as clear, but the addition of immutable lists was a valuable feature.


### Flat Versus Container Sequences

the similarities and differences between ABC compounds and Python tuples, highlighting how tuples were made to behave as sequences, contributing to Python's success. The author also introduces the concepts of container and flat sequences, and how they differ in terms of memory models. The article concludes with a discussion on the use of the term "container" and its different meanings in Python documentation.


### Mixed bag lists


In [17]:
l = [28, 14, "28", 5, "9", "1", 0, 6, "23", 19]

In [19]:
sorted(l, key=lambda x: int(x))

[0, '1', 5, 6, '9', 14, 19, '23', 28, '28']

### Key is Brilliant


In [20]:
l = [28, 14, "28", 5, "9", "1", 0, 6, "23", 19]

sorted(l, key=int)

[0, '1', 5, 6, '9', 14, 19, '23', 28, '28']

In [ ]:
sorted(l, key=str)

### Lecturers

1. Mohammad Sadegh Majidi date: 04-10-2023, [GitHub](https://github.com/cmatrix1/)
2. Mehrdad biukian date: 04-10-2023, [LinkedIn](www.linkedin.com/in/mehrdad-biukian-naeini)

#### Reviewers

1. Reza Hashemian date: 09-29-2023, [LinkedIn](https://www.linkedin.com/in/rezahashemian)
2. Fahimeh Asaadi date: 05-10-2023, [LinkedIn](https://www.linkedin.com/in/fahimehasaadi)
